In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [15]:
import pandas as pd
traf_df = pd.read_csv("data/traffic_accidents.csv")
traf_df.head()


,crash_date,traffic_control_device,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,crash_type,...,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,crash_hour,crash_day_of_week,crash_month
0,07/29/2023 01:00:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,13,7,7
1,08/13/2023 12:11:00 AM,TRAFFIC SIGNAL,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0,1,8
2,12/09/2021 10:30:00 AM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,10,5,12
3,08/09/2023 07:55:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,...,NONINCAPACITATING INJURY,5.0,0.0,0.0,5.0,0.0,0.0,19,4,8
4,08/19/2023 02:55:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,14,7,8


In [16]:
traf_target = traf_df["crash_type"] #pulling target out before dropping non-numerics

traf_features = traf_df.drop(columns=["crash_type"])

traf_features = traf_features.dropna(axis=1, how="all") # dropping empty variables col-wise
traf_features = traf_features.dropna(axis=0, how="any") #dropping rows with any missing values

traf_target = traf_target.loc[traf_features.index] #target and remaining rows aligned

traf_features = traf_features.select_dtypes(include=["number"]) #keeping only numeric cols

print(traf_features.info())
print("Features shape:", traf_features.shape)
print("Target shape:", traf_target.shape)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209306 entries, 0 to 209305
Data columns (total 10 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   num_units                      209306 non-null  int64  
 1   injuries_total                 209306 non-null  float64
 2   injuries_fatal                 209306 non-null  float64
 3   injuries_incapacitating        209306 non-null  float64
 4   injuries_non_incapacitating    209306 non-null  float64
 5   injuries_reported_not_evident  209306 non-null  float64
 6   injuries_no_indication         209306 non-null  float64
 7   crash_hour                     209306 non-null  int64  
 8   crash_day_of_week              209306 non-null  int64  
 9   crash_month                    209306 non-null  int64  
dtypes: float64(6), int64(4)
memory usage: 16.0 MB
None
Features shape: (209306, 10)
Target shape: (209306,)


In [17]:
# train/test splitting
traf_features_train, traf_features_test, traf_target_train, traf_target_test = train_test_split(
    traf_features,
    traf_target,
    test_size=0.3,        #.7 train, .3 test
    random_state=42,      
    stratify=traf_target       # keeps class proportions similar in train and test
)

print(traf_features_train.shape, traf_features_test.shape)
print(traf_target_train.shape, traf_target_test.shape)

(146514, 10) (62792, 10)
(146514,) (62792,)


In [18]:
#eval helper, maybe delete later

def evaluate_model(model_name, model, features_train, target_train, features_test, target_test):
    model.fit(features_train, target_train)
    predictions = model.predict(features_test)

    accuracy = accuracy_score(target_test, predictions)
    precision = precision_score(target_test, predictions, average="weighted", zero_division=0)
    recall = recall_score(target_test, predictions, average="weighted", zero_division=0)

    print(f"--- {model_name} ---")
    print(f"Accuracy : {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall   : {recall:.4f}")
    print()
    
    return accuracy, precision, recall

In [19]:
traf_naive_bayes = GaussianNB()
nb_results = evaluate_model(
    "Naive Bayes (Traffic)",
    traf_naive_bayes,
    traf_features_train,
    traf_target_train,
    traf_features_test,
    traf_target_test
)

traf_logistic_regression = LogisticRegression(max_iter=1000)
lr_results = evaluate_model(
    "Logistic Regression (Traffic)",
    traf_logistic_regression,
    traf_features_train,
    traf_target_train,
    traf_features_test,
    traf_target_test
)

traf_knn = KNeighborsClassifier(n_neighbors=5)
knn_results = evaluate_model(
    "Traffic KNN (Traffic, k=5)",
    traf_knn,
    traf_features_train,
    traf_target_train,
    traf_features_test,
    traf_target_test
)

traf_decision_tree = DecisionTreeClassifier(random_state=42)
dt_results = evaluate_model(
    "Decision Tree (Traffic)",
    traf_decision_tree,
    traf_features_train,
    traf_target_train,
    traf_features_test,
    traf_target_test
)

traf_mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, random_state=42)
mlp_results = evaluate_model(
    "MLP (Traffic, 50 hidden units)",
    traf_mlp,
    traf_features_train,
    traf_target_train,
    traf_features_test,
    traf_target_test
)

--- Naive Bayes (Traffic) ---
Accuracy : 0.8213
Precision: 0.8638
Recall   : 0.8213

--- Logistic Regression (Traffic) ---
Accuracy : 0.8212
Precision: 0.8636
Recall   : 0.8212

--- Traffic KNN (Traffic, k=5) ---
Accuracy : 0.7985
Precision: 0.8082
Recall   : 0.7985

--- Decision Tree (Traffic) ---
Accuracy : 0.8106
Precision: 0.8201
Recall   : 0.8106

--- MLP (Traffic, 50 hidden units) ---
Accuracy : 0.8279
Precision: 0.8535
Recall   : 0.8279



In [20]:
flight_df = pd.read_csv("data/Combined_Flights_2022.csv")
flight_df.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,...,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,...,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,...,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,GPT,False,False,1435,1430.0,0.0,-5.0,...,1446.0,1543.0,4.0,1605,-18.0,0.0,-2.0,1600-1659,2,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1135,1135.0,0.0,0.0,...,1154.0,1243.0,8.0,1245,6.0,0.0,0.0,1200-1259,2,0


In [21]:
flight_target = flight_df["Cancelled"]
flight_features = flight_df.drop(columns=["Cancelled"])

flight_features = flight_features.dropna(axis=1, how="all")

flight_features = flight_features.dropna(axis=1, how="any")

flight_features = flight_features.dropna(axis=0, how="any")
flight_target = flight_target.loc[flight_features.index]

flight_features = flight_features.select_dtypes(include=["number"])

print("Final target distribution:")
print(flight_target.value_counts(dropna=False))

Final target distribution:
Cancelled
False    3955126
True      123192
Name: count, dtype: int64


In [22]:
sample_size = 200000  # smaller because data set is huge, models take too long

flight_features_sampled = flight_features.sample(n=sample_size, random_state=42)
flight_target_sampled = flight_target.loc[flight_features_sampled.index]

print("\nSampled features shape:", flight_features_sampled.shape)
print("Sampled target distribution:")
print(flight_target_sampled.value_counts(dropna=False))

flight_features_train, flight_features_test, flight_target_train, flight_target_test = train_test_split(
    flight_features_sampled,
    flight_target_sampled,
    test_size=0.3,
    random_state=42,
    stratify=flight_target_sampled
)


Sampled features shape: (200000, 25)
Sampled target distribution:
Cancelled
False    194064
True       5936
Name: count, dtype: int64


In [23]:
flight_nb = GaussianNB()
evaluate_model(
    "Naive Bayes (Flights)",
    flight_nb,
    flight_features_train,
    flight_target_train,
    flight_features_test,
    flight_target_test
)

flight_lr = LogisticRegression(max_iter=2000)
evaluate_model(
    "Logistic Regression (Flights)",
    flight_lr,
    flight_features_train,
    flight_target_train,
    flight_features_test,
    flight_target_test
)

flight_knn = KNeighborsClassifier(n_neighbors=5)
evaluate_model(
    "KNN (Flights, k=5)",
    flight_knn,
    flight_features_train,
    flight_target_train,
    flight_features_test,
    flight_target_test
)

flight_dt = DecisionTreeClassifier(random_state=42)
evaluate_model(
    "Decision Tree (Flights)",
    flight_dt,
    flight_features_train,
    flight_target_train,
    flight_features_test,
    flight_target_test
)

flight_mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, random_state=42)
evaluate_model(
    "MLP (Flights, 50 hidden units)",
    flight_mlp,
    flight_features_train,
    flight_target_train,
    flight_features_test,
    flight_target_test
)

--- Naive Bayes (Flights) ---
Accuracy : 0.9703
Precision: 0.9415
Recall   : 0.9703



/Users/gargoyle/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


--- Logistic Regression (Flights) ---
Accuracy : 0.9703
Precision: 0.9415
Recall   : 0.9703

--- KNN (Flights, k=5) ---
Accuracy : 0.9697
Precision: 0.9464
Recall   : 0.9697

--- Decision Tree (Flights) ---
Accuracy : 0.9463
Precision: 0.9516
Recall   : 0.9463

--- MLP (Flights, 50 hidden units) ---
Accuracy : 0.9703
Precision: 0.9415
Recall   : 0.9703



(0.9703166666666667, 0.9415144336111111, 0.9703166666666667)